In [1]:
pip install fastparquet

  Using cached https://files.pythonhosted.org/packages/48/2b/0f5542c5ae98c13fdb27bf3ac57d66916e34b880c84b567105123a96abab/fas-0.1.5.tar.gz
ERROR: Could not find a version that satisfies the requirement tparquet (from versions: none)
ERROR: No matching distribution found for tparquet


In [2]:
import pandas as pd
from fastparquet import write
# 댓글 크롤링 데이터 불러옴
df=pd.read_parquet('all_comment_id.parquet',engine='fastparquet')

In [3]:
# 댓글 아이디 별로 분류, 웹툰 댓글 단 개수 값 넣어서
df = df.groupby(['아이디','제목']).size().reset_index(name='count')
df = df.pivot('아이디', '제목', 'count').fillna(0) 
print(df)
df.to_parquet('trying.parquet', compression='gzip')

제목             0.0MHz  0.1초의 설렘  100%의 그녀  ...  히트포인트  힘내요 일본!-[릴레이웹툰]  힙한남자
아이디                                        ...                              
       불렉션        0.0       0.0       0.0  ...    0.0              0.0   0.0
       찐          0.0       0.0       0.0  ...    0.0              0.0   0.0
 Lr매ㄱr고ㅍr         0.0       0.0       0.0  ...    0.0              0.0   0.0
 chahy            0.0       0.0       0.0  ...    0.0              0.0   0.0
 곰지123            0.0       0.0       0.0  ...    0.0              0.0   0.0
...               ...       ...       ...  ...    ...              ...   ...
＊ㄴδㅁξ - - ª..     0.0       0.0       0.0  ...    0.0              0.0   0.0
＊오가＊              0.0       0.0       0.0  ...    0.0              0.0   0.0
＊좋은생각＊            0.0       0.0       0.0  ...    0.0              0.0   0.0
￡0ⓥe ⓘs...        0.0       0.0       0.0  ...    0.0              0.0   0.0
￡ΟνΕ　 Ιε 。。♡      0.0       0.0       0.0  ...    0.0              0.0   0.0

In [4]:
# 비슷한 사용자 k 명 찾기
from sklearn.metrics.pairwise import cosine_similarity
import operator

def similar_users(user_id, matrix, k=5):
    # 입력받은 사용자 데이터 프레임 생성
    user = matrix[matrix.index == user_id]
    
    # 입력받은 사용자 제외 다른 사용자의 데이터 프레임 생성
    other_users = matrix[matrix.index != user_id]
    
    # 입력받은 사용자와 다른 사용자간의 코사인 유사도 측정
    similarities = cosine_similarity(user,other_users)[0].tolist()
    
    # 사용자 인덱스의 리스트 생성
    indices = other_users.index.tolist()
    
    # 사용자 인덱스와 유사도를 key,value값으로 딕셔너리 생성
    index_similarity = dict(zip(indices, similarities))
    
    # 유사도로 소팅
    index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1))
    index_similarity_sorted.reverse()
    
    # 유사한 k명의 top 사용자 생성
    top_users_similarities = index_similarity_sorted[:k]
    users = [u[0] for u in top_users_similarities]
    
    return users
    


In [5]:
#비슷한 사용자들 평균내서, 추천 웹툰 목록 만듬
def recommend_item(user_index, similar_user_indices, matrix):
    
    # 유사 사용자의 벡터 생성
    similar_users = matrix[matrix.index.isin(similar_user_indices)]
    # 5명의 유사 사용자의 평균 댓글 횟수 계산
    similar_users = similar_users.mean(axis=0)
    # 소팅과 필터링을 용이하게 하기 위한 데이터프레임 변환
    similar_users_df = pd.DataFrame(similar_users, columns=['mean'])
    
    # 현재 사용자의 벡터 생성
    user_df = matrix[matrix.index == user_index]
    # 필터링을 쉽게 하기 위해 transpose
    user_df_transposed = user_df.transpose()
    # rating column으로 이름 변경
    user_df_transposed.columns = ['rating']
    # 본 웹툰을 제외한 나머지 웹툰을 이용하여 데이터프레임 생성
    user_df_transposed = user_df_transposed[user_df_transposed['rating']==0]
    # 보지 않은 웹툰 리스트생성
    webtoon_unseen = user_df_transposed.index.tolist()
    
    # 현재 사용자가 보지 않은 웹툰 많을 이용하여 비슷한 사용자의 댓글 횟수 평균을 필터링
    similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(webtoon_unseen)]
    # 데이터 프레임 내림차순으로 소팅
    similar_users_df_ordered = similar_users_df_filtered.sort_values(by=['mean'], ascending=False)
    print(similar_users_df_ordered)
    
    return similar_users_df_ordered #items



In [11]:

# 사용자가 최근 본 웹툰 3개 여기에 입력
user_webtoon_input=['유미의 세포들','삼국지툰','놓정동화']

# 파일 읽기  
df=pd.read_parquet('trying.parquet',engine='fastparquet')
# 전체 데이터 프레임에 최근 본 웹툰 3개의 가중치값을 1로 두어 행을 추가
df.loc['user_input']={user_webtoon_input[0]:1,user_webtoon_input[1]:1,user_webtoon_input[2]:1}
df=df.fillna(0)

# 유사 사용자 받아옴    
current_user = "user_input"
similar_user_indices = similar_users(current_user, df)
print(similar_user_indices)    

['huj9****', 'kne4****', 'kysb****', 'yeps****', 'nnno****']


In [12]:
# 추천 리스트 생성
recommend_list  = recommend_item(current_user, similar_user_indices, df)
print(recommend_list)

               mean
제목                 
남기한엘리트만들기       0.4
여성전용헬스장 진달래짐    0.2
노블레스            0.2
오늘도 핸드메이드!      0.2
오늘은 꼭 고백할 거야!   0.0
...             ...
마야비             0.0
마애(磨崖)          0.0
마스크걸            0.0
마술사             0.0
힙한남자            0.0

[1821 rows x 1 columns]
               mean
제목                 
남기한엘리트만들기       0.4
여성전용헬스장 진달래짐    0.2
노블레스            0.2
오늘도 핸드메이드!      0.2
오늘은 꼭 고백할 거야!   0.0
...             ...
마야비             0.0
마애(磨崖)          0.0
마스크걸            0.0
마술사             0.0
힙한남자            0.0

[1821 rows x 1 columns]


In [13]:
recommend_list.head(10)

,mean
제목,
남기한엘리트만들기,0.4
여성전용헬스장 진달래짐,0.2
노블레스,0.2
오늘도 핸드메이드!,0.2
오늘은 꼭 고백할 거야!,0.0
오리지널 씬2,0.0
오리우리,0.0
오른눈이 보는 세계,0.0
오로지 너를 이기고 싶어,0.0
